In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import xlwings as xw
import tkinter as tk
from tkinter import *
from tkinter import filedialog

class Application(tk.Frame):
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.pack()
        self.createWidgets()

    def createWidgets(self):
        self.BPA = tk.Button(self)
        self.BPA["text"] = "Import BPA Files"
        self.BPA["command"] = self.getBPA
        self.BPA.pack(side="top")
        
        self.Todo = tk.Button(self)
        self.Todo["text"] = "Import VR File"
        self.Todo["command"] = self.getTodotable
        self.Todo.pack(side="top")
        
        self.Fillin = tk.Button(self)
        self.Fillin["text"] = "Fill in item code"
        self.Fillin["command"] = self.fillinVR
        self.Fillin.pack(side="top")
        
        self.BPAamount= tk.Button(self)
        self.BPAamount["text"] = "BPA amount"
        self.BPAamount["command"] = self.BPA_amount_caculate
        self.BPAamount.pack(side="top")


        self.QUIT = tk.Button(self, text="QUIT", fg="red",command=root.destroy)
        self.QUIT.pack(side="bottom")

    def getBPA(self):
        self.filename =  filedialog.askopenfilenames(initialdir = "/",title = "Select file",filetypes = [('Excel 97-2003 Workbook', '.xls'), ('all files', '.*'),])
        self.BPA_Files =list(self.filename)
        self.BPA_search=pd.DataFrame()
        for BPA_book in self.BPA_Files:
            BPA_temp = pd.read_excel(BPA_book, sheet_name='Sheet0')
            BPA_temp = BPA_temp.replace('nan', np.nan).dropna(how='all')
            BPA_file_full_name=os.path.basename(BPA_book)
            BPA_name_list=BPA_file_full_name.split('_')
            BPA_number=BPA_name_list[2]
            BPA_temp["BPA_number"]=BPA_number=BPA_name_list[2]
            self.BPA_search=self.BPA_search.append(BPA_temp, ignore_index=True)
        self.BPA_search['Description*']=self.BPA_search['Description*'].apply(lambda x : x.upper())
        self.BPA_search['Description*']=self.BPA_search['Description*'].apply(lambda x: x.replace(" ",""))
        self.BPA_search['Description*']=self.BPA_search['Description*'].apply(lambda x: x.replace("-",""))
        self.BPA_search['Description*']=self.BPA_search['Description*'].apply(lambda x: x.replace("–",""))
        self.BPA_search['Description*']=self.BPA_search['Description*'].apply(lambda x: x.replace("/",""))
        self.BPA_search.set_index(keys=['BPA_number','Price*','Description*'],inplace=True)
        print("BPA is imported") 
    
    def getTodotable(self):
        self.filename_workingtable =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = [('Excel Workbook', '.xlsx'), ('all files', '.*'),])
        self.Working_table = pd.read_excel(self.filename_workingtable, sheet_name='Mytodo')
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.replace("’",""))
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.upper())
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.replace(" ",""))
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.replace("–",""))
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.replace("-",""))
        self.Working_table['Description']=self.Working_table['Description'].apply(lambda x: x.replace("/",""))
        print("VR table is imported")
        
    def fillinVR(self):
        for row in range(0,len(self.Working_table)):
            try:
                BPA_number=str(self.Working_table.loc[row,'BPA number'])
                Price=self.Working_table['Unit Price'][row]
                Description=self.Working_table['Description'][row]
                item_code=self.BPA_search.loc[BPA_number,Price,Description]['Item']
                self.Working_table.loc[row,'ITEM']=item_code
            except KeyError:
                self.Working_table.loc[row,'ITEM']='#NA'
        app=xw.App(visible=True,add_book=False)
        wb=app.books.open(self.filename_workingtable)
        sht = wb.sheets[0]
        sht.range('R2').value=np.vstack(np.array(self.Working_table['ITEM']))
        print('item codes have been filled')
        
    def BPA_amount_caculate(self):
        self.BPA_Amount=self.Working_table.groupby('BPA number')[["Amount"]].sum()
        Whole_route=filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = [('Excel Workbook', '.xlsx'), ('all files', '.*'),])
        writer = pd.ExcelWriter(Whole_route+'.xlsx')
        self.BPA_Amount.to_excel(writer,'BPA')
        writer.save()
        print("BPA amount has been saved")
        

root = tk.Tk()
app = Application(master=root)
app.mainloop()

BPA is imported
VR table is imported
BPA amount has been saved
